# The first lab results will be used to predict patient diagnoses for the X most common causes of death in ICU patients recorded in the MIMIC III database.

# MIMIC-III Critical Care Database

MIMIC-III (Medical Information Mart for Intensive Care III) is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012.

The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (both in and out of hospital).

MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors:

it is freely available to researchers worldwide
it encompasses a diverse and very large population of ICU patients
it contains high temporal resolution data including lab results, electronic documentation, and bedside monitor trends and waveforms.

Citations: 
MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635

Pollard, T. J. & Johnson, A. E. W. The MIMIC-III Clinical Database http://dx.doi.org/10.13026/C2XW26 (2016).

In [1]:
import sys
import pandas as pd
import datetime as datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot as plt
#import psycopg2
from scipy.stats import ks_2samp
import scipy.stats as scats
import visuals as vs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

plt.style.use('ggplot') 
%matplotlib inline

# IMPORTING DATA
The mimic III database was downloaded and reconstructed locally using posgresql. The database was managed graphically using Portico. 
The mimic III database was queried for information regarding ICD-9 codes and corresponding plain text diagnosis for patients who died while in the hospital. Data was then imported and analyzed to determine the X most common codes/diagnoses. 


In [23]:
diagnoses_codes = pd.DataFrame.from_csv('IDC9_DEADLY_DIAGNOSES.csv')
diagnoses_codes['icustay_id'] = diagnoses_codes.index
diagnoses_codes.set_index(np.arange(diagnoses_codes.shape[0]), inplace = True)

diagnoses_codes.head()

,diagnosis,icd9_code,short_title,icustay_id
0,DYSPNEA,48241,Meth sus pneum d/t Staph,280836
1,DYSPNEA,4280,CHF NOS,280836
2,DYSPNEA,51881,Acute respiratry failure,280836
3,DYSPNEA,5845,"Ac kidny fail, tubr necr",280836
4,DYSPNEA,4275,Cardiac arrest,280836


In [5]:
diagnoses_codes.shape

(94763, 3)

In [4]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_counts = diagnoses_codes.groupby(['icd9_code', 'short_title']).count()
diagnoses_counts.shape

(3254, 1)

In [17]:
diagnoses_counts.sort(['diagnosis'], ascending = False).head()

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,,diagnosis
icd9_code,short_title,
51881,Acute respiratry failure,2668
4019,Hypertension NOS,2260
42731,Atrial fibrillation,2237
4280,CHF NOS,2195
5849,Acute kidney failure NOS,2132


In [30]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_codes_unique = diagnoses_codes.drop_duplicates('icustay_id')
diagnoses_counts3 = diagnoses_codes_unique.groupby(['diagnosis']).count()

display(diagnoses_counts3.shape)
diagnoses_counts3.head()


(2337, 3)

,icd9_code,short_title,icustay_id
diagnosis,,,
? SEROTONIN SYNDROME,1,1,1
(AML) ACUTE MYELOGENOUS LEUKEMIA,6,6,6
(AML) ACUTE MYELOGENOUS LEUKEMIA;HYPOTENSION,1,1,1
0681,1,1,1
12-AQV,1,1,1


In [31]:
diagnoses_counts3.sort(['icustay_id'], ascending = False).head(10)

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,icd9_code,short_title,icustay_id
diagnosis,,,
SEPSIS,311,311,311
PNEUMONIA,289,289,289
INTRACRANIAL HEMORRHAGE,241,241,241
CONGESTIVE HEART FAILURE,145,145,145
ALTERED MENTAL STATUS,100,100,100
ABDOMINAL PAIN,96,96,96
HYPOTENSION,90,90,90
S/P FALL,84,84,84
CARDIAC ARREST,79,79,79


In [18]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_counts3 = diagnoses_codes.groupby(['diagnosis']).count()
display(diagnoses_counts3.shape)
diagnoses_counts3.head()


(2337, 2)

,icd9_code,short_title
diagnosis,,
? SEROTONIN SYNDROME,12,12
(AML) ACUTE MYELOGENOUS LEUKEMIA,106,106
(AML) ACUTE MYELOGENOUS LEUKEMIA;HYPOTENSION,25,25
0681,6,6
12-AQV,27,27


## The number of patients admitted with a given diagnoses and who passed away while in the hospital was calculated for all diagnoses. The results indicate that Sepsis, Pneumonia, Congestive Heart Failure and Intracranial Hemorrhage were the 4 most common, deadly diagnoses. 

In [20]:
diagnoses_counts3.sort(['icd9_code'], ascending = False).head(10)

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,icd9_code,short_title
diagnosis,,
SEPSIS,5145,5145
PNEUMONIA,4497,4497
CONGESTIVE HEART FAILURE,2379,2379
INTRACRANIAL HEMORRHAGE,2138,2138
ALTERED MENTAL STATUS,1706,1706
ABDOMINAL PAIN,1683,1683
HYPOTENSION,1536,1536
FEVER,1066,1066
S/P FALL,1045,1045


In [ ]:
diagnoses_list = diagnoses_codes.icd9_code.unique()
diagnoses_list2 = diagnoses_codes.diagnosis.unique()
display(len(diagnoses_list))
display(len(diagnoses_list2))
unique_codes = diagnoses_codes.drop_duplicates(['icd9_code', 'short_title'])
unique_codes.shape

diag_counts = diagnoses_codes.icd9_code.value_counts()
diag_counts[0:10]
diagnoses_list[0:10]

## The database was queried to find initial lab results for patients diagnosed with any of the four previously identified diagnoses.  

In [21]:
lab_results = pd.DataFrame.from_csv('DEADLY_DIAGNOSES.csv')
lab_results.groupby('diagnosis').count()

,hadm_id,icustay_id,mort_icu,mort_hosp,diagnosis,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
subject_id,,,,,,,,,,,,,,,,,,,,,
21,109451,217847,0,0,CONGESTIVE HEART FAILURE,26.0,NaN,15.0,0.4,3.7,...,2.0,5.9,261.0,5.0,150.0,4.0,23.5,140.0,94.0,23.5
21,111970,216859,1,1,SEPSIS,19.0,2.1,23.0,0.3,4.2,...,1.5,4.4,275.0,3.8,31.7,1.7,18.1,137.0,49.0,30.5
37,188670,213503,0,0,CONGESTIVE HEART FAILURE,10.0,NaN,33.0,NaN,1.0,...,2.2,3.9,263.0,4.0,24.6,1.2,13.3,139.0,37.0,10.4
42,119203,210828,0,0,CONGESTIVE HEART FAILURE,14.0,3.6,25.0,0.3,0.7,...,1.9,2.8,228.0,4.3,25.7,1.6,15.4,139.0,18.0,7.9
68,170467,225771,0,0,PNEUMONIA,15.0,NaN,22.0,NaN,2.2,...,2.1,3.8,95.0,3.4,NaN,NaN,NaN,134.0,32.0,1.9


## Perform count of the number of lab results that exist per diagnosis

In [32]:
lab_results2 = lab_results.groupby(['diagnosis']).count()
display(lab_results2.shape)
lab_results2.head(10)

(4, 24)

,hadm_id,icustay_id,mort_icu,mort_hosp,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,chloride_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
diagnosis,,,,,,,,,,,,,,,,,,,,,
CONGESTIVE HEART FAILURE,1025,1025,1025,1025,1005,330,1011,406,1013,1013,...,986,941,1010,1013,897,899,899,1013,1012,1010
INTRACRANIAL HEMORRHAGE,754,754,754,754,717,243,718,254,735,732,...,684,681,732,734,712,714,715,732,735,732
PNEUMONIA,1662,1662,1662,1662,1646,671,1649,866,1651,1652,...,1604,1588,1648,1652,1387,1400,1400,1652,1651,1649
SEPSIS,1290,1290,1290,1290,1275,824,1275,1000,1277,1279,...,1257,1256,1275,1279,1185,1189,1189,1279,1277,1274


## Many of the lab recordings were unavailable and are entered as NaN. The method below utilizes the Imputer function.
The imputer function can extrapolate missing values from existing values a number of ways. The 'mean' method replaces missing values with the calculated mean for the column. We will be a bit more targeted in that we would like the missing values in a given category of diagnosis to be replaced by the mean of the values within that category. The dataframe will be split and imputed, then reconstructed. 

In [34]:

cong_hf = lab_results[lab_results['diagnosis'] == 'CONGESTIVE HEART FAILURE']
cong_hf.head()

,hadm_id,icustay_id,mort_icu,mort_hosp,diagnosis,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
subject_id,,,,,,,,,,,,,,,,,,,,,
21,109451,217847,0,0,CONGESTIVE HEART FAILURE,26.0,NaN,15.0,0.4,3.7,...,2.0,5.9,261.0,5.0,150.0,4.0,23.5,140.0,94.0,23.5
37,188670,213503,0,0,CONGESTIVE HEART FAILURE,10.0,NaN,33.0,NaN,1.0,...,2.2,3.9,263.0,4.0,24.6,1.2,13.3,139.0,37.0,10.4
42,119203,210828,0,0,CONGESTIVE HEART FAILURE,14.0,3.6,25.0,0.3,0.7,...,1.9,2.8,228.0,4.3,25.7,1.6,15.4,139.0,18.0,7.9
164,182743,218740,1,1,CONGESTIVE HEART FAILURE,14.0,NaN,26.0,NaN,0.6,...,1.8,3.4,145.0,4.2,46.1,7.1,33.5,138.0,25.0,16.4
305,122211,224571,0,0,CONGESTIVE HEART FAILURE,10.0,NaN,29.0,NaN,0.7,...,1.9,3.6,259.0,3.7,NaN,NaN,NaN,141.0,19.0,4.4


In [66]:
lab_results.diagnosis.unique()

array(['CONGESTIVE HEART FAILURE', 'SEPSIS', 'PNEUMONIA',
       'INTRACRANIAL HEMORRHAGE'], dtype=object)

In [67]:
cong_hf_features = lab_results[lab_results['diagnosis'] == 'CONGESTIVE HEART FAILURE'][lab_results.columns[5:]]
cong_hf_labels = lab_results[lab_results['diagnosis'] == 'CONGESTIVE HEART FAILURE'][lab_results.columns[:5]]
seps_features = lab_results[lab_results['diagnosis'] == 'SEPSIS'][lab_results.columns[5:]]
seps_labels = lab_results[lab_results['diagnosis'] == 'SEPSIS'][lab_results.columns[:5]]
pneum_features = lab_results[lab_results['diagnosis'] == 'PNEUMONIA'][lab_results.columns[5:]]
pneum_labels = lab_results[lab_results['diagnosis'] == 'PNEUMONIA'][lab_results.columns[:5]]
intracran_features = lab_results[lab_results['diagnosis'] == 'INTRACRANIAL HEMORRHAGE'][lab_results.columns[5:]]
intracran_labels = lab_results[lab_results['diagnosis'] == 'INTRACRANIAL HEMORRHAGE'][lab_results.columns[:5]]
display(cong_hf_features.shape)
display(seps_features.shape)
display(pneum_features.shape)
display(intracran_features.shape)

(1025, 20)

(1290, 20)

(1662, 20)

(754, 20)

## Example code for how to impute dataframe by column

In [53]:
cong_hf_features2 = cong_hf_features.copy()
cong_hf_features2['aniongap_1st'] = imp.fit_transform(cong_hf_features[['aniongap_1st']]).ravel()


In [55]:
cong_hf_features2['aniongap_1st'].isnull().sum()

0

## Impute whole dataframe

In [71]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
cong_hf_imp = pd.DataFrame(imp.fit_transform(cong_hf_features))
cong_hf_imp.columns = cong_hf_features.columns
cong_hf_imp.index = cong_hf_features.index
seps_imp = pd.DataFrame(imp.fit_transform(seps_features))
seps_imp.columns = seps_features.columns
seps_imp.index = seps_features.index
pneum_imp = pd.DataFrame(imp.fit_transform(pneum_features))
pneum_imp.columns = pneum_features.columns
pneum_imp.index = pneum_features.index
intracran_imp = pd.DataFrame(imp.fit_transform(intracran_features))
intracran_imp.columns = intracran_features.columns
intracran_imp.index = intracran_features.index

In [48]:
cong_hf_imp.isnull().sum()

aniongap_1st       0
albumin_1st        0
bicarbonate_1st    0
bilirubin_1st      0
creatinine_1st     0
chloride_1st       0
glucose_1st        0
hematocrit_1st     0
hemoglobin_1st     0
lactate_1st        0
magnesium_1st      0
phosphate_1st      0
platelet_1st       0
potassium_1st      0
ptt_1st            0
inr_1st            0
pt_1st             0
sodium_1st         0
bun_1st            0
wbc_1st            0
dtype: int64

In [72]:
cong_hf_imputed = pd.concat([cong_hf_labels, cong_hf_imp], axis=1, join_axes=[cong_hf_labels.index])
seps_imputed = pd.concat([seps_labels, seps_imp], axis = 1, join_axes = [seps_labels.index])
pneum_imputed = pd.concat([pneum_labels, pneum_imp], axis = 1, join_axes = [pneum_labels.index])
intracran_imputed = pd.concat([intracran_labels, intracran_imp], axis = 1, join_axes = [intracran_labels.index])

In [77]:
cong_hf_imputed.columns == seps_imputed.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [76]:
seps_imputed.columns ==  pneum_imputed.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [78]:
pneum_imputed.columns == intracran_imputed.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [79]:
data_imputed = pd.concat([cong_hf_imputed, seps_imputed, pneum_imputed, intracran_imputed])
data_imputed.head()

,hadm_id,icustay_id,mort_icu,mort_hosp,diagnosis,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
subject_id,,,,,,,,,,,,,,,,,,,,,
21,109451,217847,0,0,CONGESTIVE HEART FAILURE,26.0,3.400606,15.0,0.400000,3.7,...,2.0,5.9,261.0,5.0,150.000000,4.000000,23.500000,140.0,94.0,23.5
37,188670,213503,0,0,CONGESTIVE HEART FAILURE,10.0,3.400606,33.0,0.888916,1.0,...,2.2,3.9,263.0,4.0,24.600000,1.200000,13.300000,139.0,37.0,10.4
42,119203,210828,0,0,CONGESTIVE HEART FAILURE,14.0,3.600000,25.0,0.300000,0.7,...,1.9,2.8,228.0,4.3,25.700000,1.600000,15.400000,139.0,18.0,7.9
164,182743,218740,1,1,CONGESTIVE HEART FAILURE,14.0,3.400606,26.0,0.888916,0.6,...,1.8,3.4,145.0,4.2,46.100000,7.100000,33.500000,138.0,25.0,16.4
305,122211,224571,0,0,CONGESTIVE HEART FAILURE,10.0,3.400606,29.0,0.888916,0.7,...,1.9,3.6,259.0,3.7,39.892754,1.797453,18.313793,141.0,19.0,4.4


In [84]:
data_imputed.isnull().sum()

hadm_id            0
icustay_id         0
mort_icu           0
mort_hosp          0
diagnosis          0
aniongap_1st       0
albumin_1st        0
bicarbonate_1st    0
bilirubin_1st      0
creatinine_1st     0
chloride_1st       0
glucose_1st        0
hematocrit_1st     0
hemoglobin_1st     0
lactate_1st        0
magnesium_1st      0
phosphate_1st      0
platelet_1st       0
potassium_1st      0
ptt_1st            0
inr_1st            0
pt_1st             0
sodium_1st         0
bun_1st            0
wbc_1st            0
dtype: int64

In [83]:
lab_results.isnull().sum()

hadm_id               0
icustay_id            0
mort_icu              0
mort_hosp             0
diagnosis             0
aniongap_1st         88
albumin_1st        2663
bicarbonate_1st      78
bilirubin_1st      2205
creatinine_1st       55
chloride_1st         55
glucose_1st          57
hematocrit_1st       62
hemoglobin_1st       64
lactate_1st        1186
magnesium_1st       200
phosphate_1st       265
platelet_1st         66
potassium_1st        53
ptt_1st             550
inr_1st             529
pt_1st              528
sodium_1st           55
bun_1st              56
wbc_1st              66
dtype: int64